# Analysis of the `iqs2` dataset  


We'll now use from original BEST paper

Data taken from `BESTexample-original.R` in `BEST.zip`
via https://web.archive.org/web/20170708173718/https://www.indiana.edu/~kruschke/BEST/

Steps following Matti Vuorre's [blog post](https://mvuorre.github.io/posts/2017-01-02-how-to-compare-two-groups-with-robust-bayesian-estimation-using-r-stan-and-brms/) see also [src](https://github.com/mvuorre/mvuorre.github.io/tree/main/posts/2017-01-02-how-to-compare-two-groups-with-robust-bayesian-estimation-using-r-stan-and-brms) notebook.

### Data

In [1]:
iqs2 = pd.read_csv("../datasets/iqs2.csv")
iqs2.groupby("group").describe()

NameError: name 'pd' is not defined

In [ ]:
sns.histplot(data=iqs2, x="iq", hue="group");

### BEST model 1: un-shifted exponential prior on nu

In [ ]:
import bambi as bmb
import pymc as pm

#######################################################
formula1 = bmb.Formula("iq ~ 0 + group",
                       "sigma ~ 0 + group")

iqs_mean, iqs_std = iqs2["iq"].mean(), iqs2["iq"].std()
sigma_low = np.log(iqs_std / 1000)
sigma_up = np.log(iqs_std * 1000)

priors1 = {
    "group": bmb.Prior("Normal", mu=iqs_mean, sigma=1000*iqs_std),
    "sigma": {"group": bmb.Prior("Uniform", lower=sigma_low, upper=sigma_up)},
    "nu": bmb.Prior("Exponential", lam=1/29),
}


# Build model
mod1 = bmb.Model(formula=formula1,
                 family="t",
                 priors=priors1,
                 data=iqs2)
mod1

In [ ]:
mod1.build()
mod1.backend.model

In [ ]:
mod1.graph()

In [ ]:
idata1 = mod1.fit(draws=10000)

In [ ]:
az.plot_trace(idata1);

In [ ]:
# Calculate relevant quantities

# Difference posterior of the difference between means
post1 = idata1["posterior"]

post1_dmeans = post1["group"][:,:,1] - post1["group"][:,:,0]
# ALT. post1["group"].sel(group_dim="treat") - post1["group"].sel(group_dim="ctrl")
post1["dmeans"] = post1_dmeans

# Sigmas from log-sigmas
post1["sigma_treat"] = np.exp(post1["sigma_group"][:,:,1])
post1["sigma_ctrl"] = np.exp(post1["sigma_group"][:,:,0])

# # log-nu from nu
post1["log_nu"] = np.log10(post1["nu"])

# Difference in standard deviations
post1["dstd"] = post1["sigma_treat"] - post1["sigma_ctrl"]

# Effect size
var_pooled = (post1["sigma_treat"]**2 + post1["sigma_ctrl"]**2) / 2
post1["cohend"] = post1["dmeans"] / np.sqrt(var_pooled)

In [ ]:
import arviz as az
az.summary(idata1, kind="stats", hdi_prob=0.95)

In [ ]:
az.plot_posterior(idata1, round_to=4, hdi_prob=0.95, point_estimate="mode");

## BEST model from external library

In [ ]:
(1/1000) ** 2 == 0.000001

In [ ]:
import best

treated = iqs2[iqs2["group"]=="treat"]["iq"].values
controls = iqs2[iqs2["group"]=="ctrl"]["iq"].values
best_out = best.analyze_two(treated, controls, version="v1", n_samples=10000)
best_out

In [ ]:
# best.plot_all(best_out);

### BEST model 2: shifted exponential prior on nu

In [ ]:
import bambi as bmb
import pymc as pm

#######################################################
formula1 = bmb.Formula("iq ~ 0 + group",
                       "sigma ~ 0 + group")


iqs_mean, iqs_std = iqs2["iq"].mean(), iqs2["iq"].std()
sigma_low = np.log(iqs_std / 1000)
sigma_up = np.log(iqs_std * 1000)

def TruncatedExponential(name, lam, *args, dims=None, **kwargs):
    exp = pm.Exponential.dist(lam=lam)
    return pm.Truncated(name, exp, lower=1, *args, dims=dims, **kwargs)

priors2 = {
    "group": bmb.Prior("Normal", mu=iqs_mean, sigma=1000*iqs_std),
    "sigma": {"group": bmb.Prior("Uniform", lower=sigma_low, upper=sigma_up)},
    "nu": bmb.Prior("TruncatedExponential", lam=1/29, dist=TruncatedExponential),
}

# Build model
mod2 = bmb.Model(formula=formula1,
                 family="t",
                 priors=priors2,
                 data=iqs2)
mod2

In [ ]:
mod2.build()
mod2.backend.model

In [ ]:
#mod2.plot_priors(var_names=["nu"])

In [ ]:
mod2.graph()

In [ ]:
idata2 = mod2.fit(draws=10000)

In [ ]:
az.plot_trace(idata1);

In [ ]:
# Calculate relevant quantities

# Difference posterior of the difference between means
post2 = idata2["posterior"]

post2_dmeans = post2["group"][:,:,1] - post1["group"][:,:,0]
# ALT. post2["group"].sel(group_dim="treat") - post2["group"].sel(group_dim="ctrl")
post2["dmeans"] = post2_dmeans

# Sigmas from log-sigmas
post2["sigma_treat"] = np.exp(post2["sigma_group"][:,:,1])
post2["sigma_ctrl"] = np.exp(post2["sigma_group"][:,:,0])

# # log-nu from nu
post2["log_nu"] = np.log10(post2["nu"])

# Difference in standard deviations
post2["dstd"] = post2["sigma_treat"] - post2["sigma_ctrl"]

# Effect size
var_pooled = (post2["sigma_treat"]**2 + post2["sigma_ctrl"]**2) / 2
post2["cohend"] = post2["dmeans"] / np.sqrt(var_pooled)

In [ ]:
import arviz as az
az.summary(idata2, kind="stats", hdi_prob=0.95)

In [ ]:
az.plot_posterior(idata2, round_to=3, hdi_prob=0.95, point_estimate="mode");

## Comparison

In [ ]:
# az.plot_forest([idata1,idata2], model_names=["unshifted", "shifted"], combined=True,
#               var_names=["dmeans", "dstd", "cohend", "nu"])

### Other analyses

#### Equal variances t-test

In [ ]:
from scipy.stats import ttest_ind

treated = iqs2[iqs2["group"]=="treat"]["iq"].values
controls = iqs2[iqs2["group"]=="ctrl"]["iq"].values

res_eqvar = ttest_ind(treated, controls, equal_var=True)
res_eqvar.statistic, res_eqvar.pvalue

In [ ]:
ci_eqvar = res_eqvar.confidence_interval(confidence_level=0.95)
[ci_eqvar.low, ci_eqvar.high]

#### Equivalent results using a linear model

In [ ]:
import statsmodels.formula.api as smf
res_ols = smf.ols("iq ~ 1 + C(group)", data=iqs2).fit()
res_ols.tvalues["C(group)[T.treat]"], res_ols.pvalues["C(group)[T.treat]"]

In [ ]:
res_ols.conf_int().loc["C(group)[T.treat]",:].values

#### Unequal variances t-test

In [ ]:
res_uneqvar = ttest_ind(treated, controls, equal_var=False)
res_uneqvar.statistic, res_uneqvar.pvalue

In [ ]:
ci_uneqvar = res_uneqvar.confidence_interval(confidence_level=0.95)
[ci_uneqvar.low, ci_uneqvar.high]

#### Equivalent results using a linear model with unequal variances

Using generalized least squares to reproduce the unequal variance case.

In [ ]:
n_t, var_t = len(treated), treated.var(ddof=1)
n_c, var_c = len(controls), controls.var(ddof=1)
sigma2s = [var_t]*n_t + [var_c]*n_c

res_gls = smf.gls("iq ~ 1 + C(group)", data=iqs2, sigma=sigma2s).fit()
res_gls.tvalues["C(group)[T.treat]"], res_gls.pvalues["C(group)[T.treat]"]

In [ ]:
res_gls.conf_int().loc["C(group)[T.treat]",:].values

#### Bayesian equal variances model

In [ ]:
import bambi as bmb

mod_eqvar = bmb.Model("iq ~ 1 + group", data=iqs2)
mod_eqvar

In [ ]:
mod_eqvar.build()
mod_eqvar.backend.model

In [ ]:
idata_eqvar = mod_eqvar.fit(draws=2000)

In [ ]:
import arviz as az

az.summary(idata_eqvar, kind="stats", hdi_prob=0.95)

#### Bayesian unequal variances model

In [ ]:
formula = bmb.Formula("iq ~ 1 + group",
                      "sigma ~ 1 + group")
mod_uneqvar = bmb.Model(formula, data=iqs2)
print(mod_uneqvar)

In [ ]:
mod_uneqvar.build()
mod_uneqvar.backend.model

In [ ]:
idata_uneqvar = mod_uneqvar.fit(draws=2000)

In [ ]:
az.summary(idata_uneqvar, kind="stats", hdi_prob=0.95)

### Robust Bayesian Estimation

In [ ]:
formula = bmb.Formula("iq ~ 1 + group",
                      "sigma ~ group")
mod_robust = bmb.Model(formula, family="t", data=iqs2)
print(mod_robust)

In [ ]:
mod_robust.build()
# mod_robust.backend.model
mod_robust.graph()

In [ ]:
idata_robust = mod_robust.fit(draws=1000)

In [ ]:
az.summary(idata_robust, kind="stats", hdi_prob=0.95)

In [ ]:
az.plot_posterior(idata_robust);

In [ ]:
# p-value
postD = idata_robust["posterior"]["group"][0].values.flatten()
np.sum(postD < 0) / len(postD)

In [ ]:
# from scipy.stats import expon
# rvE = expon(scale=29, loc=1)
# xs = np.linspace(0, 200 ,1000)
# sns.lineplot(x=xs, y=rvE.pdf(xs))

### Robust Bayesian estimation without intercept

In [ ]:
5*np.sqrt((np.var(treated,ddof=1) + np.var(controls,ddof=1))/2)

In [ ]:
5*iqs2["iq"].std(ddof=0)

In [ ]:
formula = bmb.Formula("iq ~ 0 + group",
                      "sigma ~ 0 + group")
mod_robust2 = bmb.Model(formula, family="t", data=iqs2)
print(mod_robust2)

In [ ]:
mod_robust2.build()
# mod_robust2.backend.model
mod_robust2.graph()

In [ ]:
idata_robust2 = mod_robust2.fit(draws=2000)

post_robust2 = idata_robust2["posterior"]
dmeans_robust2 = post_robust2["group"][:,:,1] - post_robust2["group"][:,:,0]
# ALT. post_robust2["group"].sel(group_dim="treat") - post_robust2["group"].sel(group_dim="ctrl")
post_robust2["dmeans"] = dmeans_robust2

In [ ]:
az.summary(idata_robust2, kind="stats", hdi_prob=0.95)

In [ ]:
az.plot_posterior(idata_robust);